In [ ]:
import datetime
import time
import gensim
from gensim.matutils import argsort
from gensim.models.coherencemodel import CoherenceModel
from gensim.topic_coherence import segmentation
from gensim.topic_coherence import probability_estimation
from gensim.topic_coherence import direct_confirmation_measure
from gensim.topic_coherence.aggregation import arithmetic_mean
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from yappyChuck import Client

client = Client()

In [ ]:
df = pd.read_csv('../enmi.csv')

Stop list words

In [ ]:
stoplist = stopwords.words('english')
stoplist += ['enmi18'] #remove the main hashtag as it appears everywhere
stoplist += ['rt', 'dm', 'ht']#remove Twitter specific language
stoplist += stopwords.words('french')
vect = CountVectorizer(stop_words=stoplist, 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')

In [ ]:
X = vect.fit_transform(df.text)
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

In [ ]:
number_of_topics = 5
model = gensim.models.LdaModel(corpus, id2word = dict((v, k) for k, v in vect.vocabulary_.items()), 
                               num_topics=number_of_topics)

### Sonify the Topic Models

Before we can sonify the models, let's print them out.  

In [ ]:
for idx, topic_words in model.show_topics():
    topic_word = ','.join([k.split('*')[1] for k in topic_words.split('+')])
    print("{} -> {}".format(idx, topic_word))

In [ ]:
#@todo: put together a DSL for the sound
# hack to allow either a frequency or a sample
# sample -> { sample: name of file, vol:volume , dur:sample_speed }
# frequency -> { freq: float, vol: volume, dur: duration }
topics = {}
topics[1] = {'freq':130.24, 'vol': 0.5, 'dur':0.4 }
topics[3] = {'freq':224.32, 'vol': 0.4, 'dur':0.2 }

In [ ]:
#tf = tfidf_vectorizer.fit_transform(df['cleaner_text'])
#doc_topic = lda.transform(tf)
print(topics)

#make the assumption that a user may not want to sonify all topics. 
for tops in model.get_document_topics(corpus):
    for topic in tops:
        if topic[0] in topics:
            n = topic[0]
            #do something
            if "sample" in topics and topics[n]['sample'] != '':
                client.send("sample:{}:{}:{}:{}".format(topics[n]['sample'], topics[n]['vol'], topics[n]['dur'], topics[n]['speed']))
            elif "freq" in topics[n]:
                #test for freq. If exists, then use. 
                client.send("topic:{}:{}:{}".format(topics[n]['freq'] + topic[1], topics[n]['vol'] + topic[1], topics[n]['dur'] + topic[1]))

    time.sleep(1)